# Wconcept 크롤링



In [3]:
######################################################### PART 1 ########################################################
from selenium import webdriver  #브라우저 버전 맞춰주는것!
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By #어떤 방식의 명령어로 찾아올 것인가? -> By ID값으로 많이 크롤링 없으면, CSS -> XPATH -> tag 안정성 순
from webdriver_manager.chrome import ChromeDriverManager
import time, re # -> 단일 코드 두개를 한줄로 쓰는것도 가능!
import pandas as pd

##### 크롤러 준비요소
service = Service(ChromeDriverManager().install())

options = Options()
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu") # GPU 렌더링 비활성화(옛날 headless 버그 회피).
options.add_argument("--disable-dev-shm-usage") # 리뷰가 많을 때 임시메모리에 저장 못함. 우리 로컬컴퓨터의 메모리를 사용할 수 있게 해주는것.
options.add_argument("--start-maximized")  # 이게 있어야 반응형 페이지로 잡힘(?)
options.add_argument("--window-size=1920x1080")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") # 언어 한국어로 맞춰줌!

driver = webdriver.Chrome(service=service, options=options) #프로토타입객체 : 대문자로 시작하는 객체 뒤에 () 있으면. 메뉴얼을 class로만들어 놓고 이를 쓸떄는 프로토타입객체를 거쳐서 생성되고 이를 통해 생성된 것을 instant라고 불림

### 반복 순회하여 , 의류 가방 신발 액세서리 → 카테고리로 순환할것! -> 키값에 따라 반복순회하기 편하다 = 딕셔너리
category_urls = {
    "여성의류": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10101&displaySubCategoryType=ALL&gnbType=Y",
    "가방": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10102&displaySubCategoryType=ALL&gnbType=Y",
    "신발": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10103&displaySubCategoryType=ALL&gnbType=Y",
    "액세서리": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10104&displaySubCategoryType=ALL&gnbType=Y",    
} #위 4개의 카테고리들의 url을 넣어줌

### 크롤링 되어진 데이터를 담아 놓을 수 있는 변수 먼저 만들어 놓기
all_data = [] # 밖에서 값을 처리해줘야함. 반복문 안에 넣으면 값이 갇힘

# 리뷰를 찾아오려면 썸네일에서 들어가야함. BUT 검사를 해도 a태그의 URL이 보이지 않음 
### 함수를 만들어서 파라미터값을 바꿔서 URL을 자동으로 만들어주는 함수 만들기 -> img의 src에서 가져옴!
### 파라미터 뒤에 들어갈 값을 반환하는 함수 만들기
def extract_product_id_from_imgsrc(src:str) -> str | None: #인자값을 파라미터 값으로 정의 (src는 문자열이야), 문자열로 반환할거야. 만약, 값을 도출할 수 없는 상태(error)라면, 일단 None이라고 해줘
    if not src: #예외조항처리 : 만약 src의 값이 없다면(not)
        return None # 위에서 출력하라고 한 None을 출력해라
    ### src에서 원하는 문장만 찾아오는 방법 -> 정규표현식밖에 답이 없음.. 
    m = re.search(r"/(\d+)(?:[_\.][^/?]+)?(?:\?.*)?$",src) 
    # re(gular expression)에서 찾아와서 m에 값을 넣어라 / 정수를 찾아오라는 단순 문자열임이 아니고 정규표현식의 문법으로 인식하게 하기위한 'r' 
    # r은 정규식 특수문자를 이스케이프 없이 쓰기 위함 / src 안에 넣어라! (?:) 숫자가 나온 뒤에 _까지는 찾아올 필요는 없음, 그 값 뒤에는 특수문자_이거나(\) .이 붙을 수 있다
    # 숫자 뒤에 _나 .은 올 수 있으나 /나 ?는 하나이상 오는 것은 아니야. 숫자를 기준으로 언더바나 온점은 올 수 있으나, 슬래시나 물음표는 없음!
    # [] 안에서는 ^가 맨 앞에 올 경우 "not" 의미. [] 밖에서 쓰면 "문자열 시작" 으로 인식
    # 괄호 닫은 뒤에 '?'는 있을 수도 있고 없을 수도 있음을 의미! 괄호 안에 ?는 진짜 문자열 ? 임.. 
    # '^'은 [] 안에 있으면 무조건 이렇게 시작해야된다. BUT ^가 패턴에 쓰이면 not의 의미
    # ? 뒤의 () -> 찾기는 찾되 나중에 가져오지는 마! = '?:'라는 하나의 문법 / 그 뒤의 "\?"는 문자열의 ?로 인식하라고 하는 것!
    # ? 뒤의 어떠한 문자가 올때, 몇글자이던 상관이없다(갯수와 제한이 없음) '.*'
    # 그룹 바깥에 쓰여진 ? -> 있을 수도 있고 없을 수도 있다.
    # 특정 문자열의 마지막에 위치한다 => '$'
    
    ## 위와 같은 패턴을 가진 문자열을 찾아와서!((\d+)과 매칭되는것) m에 그룹화하는 것으로 반환해라! 만약(if) 없으면(else) None을 반환해라!
    return m.group(1) if m else None

### 두번쨰 함수! 완전한 url 만들기
def build_product_url(pid:str | None) -> str: # 여기서는 정규표현식의 or(\)과 다름 =|
    return f"https://www.wconcept.co.kr/Product/{pid}" if pid else "" #pid가 있다면 문자열을 반환해라, 그렇지 않다면 빈문자를 반환해라
    
### category_urls를 한번씩 반복해서 돌릴것!
for cat_name, url in category_urls.items() : # 우리는 딕셔너리를 가져올 것이기 때문에 나눠줄 필요가 있음 이떄 사용하는 함수 .items
# cat_name => 키값 , url => 밸류값 이 각각 들어감
    driver.get(url)
    time.sleep(3)

    ##중첩 for문!
    ## 사이트의 방식이 최하단으로 가면 새로운 값을 주는 사이트에서 쓰이는 반복문 ( 반복해서 스크롤 내려야함 )
    for _ in range(10) : # 값을 찾아올 필요가 없음
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") # 높이 설정 -> 매번 새로운 URL 들어갈 때마다 스크롤 내리기 / ; -> 자바스크립트의 구문 끝났다!
        time.sleep(2)

    ## 각각 카테고리마다 10번 스크롤하면서, 상품 찾아오기!
    products = driver.find_elements(By.CSS_SELECTOR,"div.product-item.item.type-all")[:100] 
    # 주의 !! 시간이 경과되면 소스코드가 바뀔 수가 있음! -> 안전하게 찾아오는 방법 : 안바뀔 것 같은것만 코드가져오기 / 100개 찾아와
    
    for product in products :
        try : # 실질적인 크롤링을하게되는 구문!
            ## 100개의 찾아온 것 중에 상품명을 제일 먼저것을 하나 찾아오고 name으로 저장해
            name = product.find_element(By.CSS_SELECTOR,"span.prdc-title span.text.detail").text.strip()# 상품명 & 브랜드 명이 필요할 수도 있으니까 prdc title에서부터 찾아옴
            try:
                img = product.find_element(By.CSS_SELECTOR,"button.area-img img") # button의area-img 안에 있는 img태그를 찾아와서
                src = img.get_attribute("src") #src만 찾아오고 싶어! . src에 넣어라
                pid = extract_product_id_from_imgsrc(src) # 위에 설정한 함수활용해서 결과값을 pid에 넣고
                url = build_product_url(pid) # 위에 설정한 pid를 넣어서 url로 완성시켜줘
            except: # 에러가 뭔지 알필요없고 발생한다면, url =""인상태로 가져와!
                url = ""
                
            ##리뷰 카운트 수 찾아오기
            #괄호를 배제하는 방법 -> 정규표현식 쓰기!
            try:
                review_info = product.find_element(By.CSS_SELECTOR,"span.area-info span.text.cnt").text
                #위에서 가져온 것에서 (10)에서 숫자 가져오기! 정규표현식으로! -> 문자열로 가져와야함(정규표현식은)
                review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0 # review_info가 있을때만! 없으면 0으로 줘라. (SQL null값 들어오는거 방지)
                ## 교체하는 정규표현식의 형태
                # 패턴인 [] 안에 정수값을 넣을것.
                #^ 숫자가 아닌값을 찾아와라
                #, "" 앞에 것을 바꿔버리자
                # int(...) : 정수값으로 형변환해야함! (정규표현식 쓰려고 문자열로 사용했으니까) 
            except:
                review_count = 0
            ## 평점가져오기
            try :
                rating_text = product.find_element(By.CSS_SELECTOR,"span.area-info em.score").text
                # 여기서 찾아온 예) 4.9 라는 숫자는 소수점을 가진 숫자값으로 바꿔줘야함 (원래 문자열)
                review_avg = float(re.findall(r"[0-9.]+", rating_text)[0]) # 0번째 값으로 찾아와
                # 정규표현식으로 찾아오는것
                # (r"[0-9.]") : 숫자 0-9까지 들어올 수 있고, '.'이 들어올 수 있음! 여기서 '.'은 숫자 뒤에 들어왔으니까 실수로 인식한다는 문법!
                # rating_text에서 찾아오면 됨!
            except: #값을 못찾아왔다면
                review_avg = 0.0 #평점 0.0으로 줘라

            ## 데이터에 값넣기!
            all_data.append({  # 딕셔너리 형태로 넣을것!
                "카테고리":cat_name,
                "상품명":name,
                "URL":url,
                "리뷰수":review_count,
                "평균평점":review_avg
            }) 
            
        except Exception as e : #Exception이 발생하면 e를 출력하고
            print("에러발생 :", e)
            continue # 다시 이어가라!

driver.quit()

print(all_data)

[{'카테고리': '여성의류', '상품명': '[72H팝업] [단독]캐시미어 피크드 칼라 더블 롱코트_2colors', 'URL': 'https://www.wconcept.co.kr/Product/306038291', '리뷰수': 87, '평균평점': 5.0}, {'카테고리': '여성의류', '상품명': '샤론 슬림 니트 가디건(멜란지그레이)', 'URL': 'https://www.wconcept.co.kr/Product/307350085', '리뷰수': 10, '평균평점': 4.9}, {'카테고리': '여성의류', '상품명': '[72H팝업] 캐시미어 발마칸 핸드메이드 코트_3colors', 'URL': 'https://www.wconcept.co.kr/Product/306038292', '리뷰수': 76, '평균평점': 4.9}, {'카테고리': '여성의류', '상품명': '[여울PICK]LOND WAIST TUCK T-SHIRT (Z-BL-4225)', 'URL': 'https://www.wconcept.co.kr/Product/305818731', '리뷰수': 1015, '평균평점': 4.9}, {'카테고리': '여성의류', '상품명': '[15%+5%쿠폰] [단독]UNISEX WAXED COTTON HUNTING JACKET_3 color', 'URL': 'https://www.wconcept.co.kr/Product/307335695', '리뷰수': 9, '평균평점': 5.0}, {'카테고리': '여성의류', '상품명': '스푸마토 언발 블라우스(카키)', 'URL': 'https://www.wconcept.co.kr/Product/307350064', '리뷰수': 5, '평균평점': 5.0}, {'카테고리': '여성의류', '상품명': '[72H팝업] 캐시미어 울 블랜디드 싱글 하프 코트_3colors', 'URL': 'https://www.wconcept.co.kr/Product/306141668', '리뷰수': 96, '평균평점': 4.8}, 

# Pandas로 값 추출해보기!

In [7]:
from selenium import webdriver  #브라우저 버전 맞춰주는것!
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By #어떤 방식의 명령어로 찾아올 것인가? -> By ID값으로 많이 크롤링 없으면, CSS -> XPATH -> tag 안정성 순
from webdriver_manager.chrome import ChromeDriverManager
import time, re # -> 단일 코드 두개를 한줄로 쓰는것도 가능!
import pandas as pd

##### 크롤러 준비요소
service = Service(ChromeDriverManager().install())

options = Options()
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu") # GPU 렌더링 비활성화(옛날 headless 버그 회피).
options.add_argument("--disable-dev-shm-usage") # 리뷰가 많을 때 임시메모리에 저장 못함. 우리 로컬컴퓨터의 메모리를 사용할 수 있게 해주는것.
options.add_argument("--start-maximized")  # 이게 있어야 반응형 페이지로 잡힘(?)
options.add_argument("--window-size=1920x1080")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR") # 언어 한국어로 맞춰줌!

driver = webdriver.Chrome(service=service, options=options) #프로토타입객체 : 대문자로 시작하는 객체 뒤에 () 있으면. 메뉴얼을 class로만들어 놓고 이를 쓸떄는 프로토타입객체를 거쳐서 생성되고 이를 통해 생성된 것을 instant라고 불림

### 반복 순회하여 , 의류 가방 신발 액세서리 → 카테고리로 순환할것! -> 키값에 따라 반복순회하기 편하다 = 딕셔너리
category_urls = {
    "여성의류": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10101&displaySubCategoryType=ALL&gnbType=Y",
    "가방": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10102&displaySubCategoryType=ALL&gnbType=Y",
    "신발": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10103&displaySubCategoryType=ALL&gnbType=Y",
    "액세서리": "https://display.wconcept.co.kr/rn/best?displayCategoryType=10104&displaySubCategoryType=ALL&gnbType=Y",    
} #위 4개의 카테고리들의 url을 넣어줌

### 크롤링 되어진 데이터를 담아 놓을 수 있는 변수 먼저 만들어 놓기
all_data = [] # 밖에서 값을 처리해줘야함. 반복문 안에 넣으면 값이 갇힘

# 리뷰를 찾아오려면 썸네일에서 들어가야함. BUT 검사를 해도 a태그의 URL이 보이지 않음 
## 함수를 만들어서 파라미터값을 바꿔서 URL을 자동으로 만들어주는 함수 만들기 -> img의 src에서 가져옴!
### 파라미터 뒤에 들어갈 값을 반환하는 함수 만들기
def extract_product_id_from_imgsrc(src:str) -> str | None: #인자값을 파라미터 값으로 정의 (src는 문자열이야), 문자열로 반환할거야. 만약, 값을 도출할 수 없는 상태(error)라면, 일단 None이라고 해줘
    if not src: #예외조항처리 : 만약 src의 값이 없다면(not)
        return None # 위에서 출력하라고 한 None을 출력해라
        
    ### src에서 원하는 문장만 찾아오는 방법 -> 정규표현식밖에 답이 없음.. 
    m = re.search(r"/(\d+)(?:[_\.][^/?]+)?(?:\?.*)?$",src) 
    # re(gular expression)에서 찾아와서 m에 값을 넣어라 / 정수를 찾아오라는 단순 문자열임이 아니고 정규표현식의 문법으로 인식하게 하기위한 'r' 
    # r은 정규식 특수문자를 이스케이프 없이 쓰기 위함 / src 안에 넣어라! (?:) 숫자가 나온 뒤에 _까지는 찾아올 필요는 없음, 그 값 뒤에는 특수문자_이거나(\) .이 붙을 수 있다
    # 숫자 뒤에 _나 .은 올 수 있으나 /나 ?는 하나이상 오는 것은 아니야. 숫자를 기준으로 언더바나 온점은 올 수 있으나, 슬래시나 물음표는 없음!
    # [] 안에서는 ^가 맨 앞에 올 경우 "not" 의미. [] 밖에서 쓰면 "문자열 시작" 으로 인식
    # 괄호 닫은 뒤에 '?'는 있을 수도 있고 없을 수도 있음을 의미! 괄호 안에 ?는 진짜 문자열 ? 임.. 
    # '^'은 [] 안에 있으면 무조건 이렇게 시작해야된다. BUT ^가 패턴에 쓰이면 not의 의미
    # ? 뒤의 () -> 찾기는 찾되 나중에 가져오지는 마! = '?:'라는 하나의 문법 / 그 뒤의 "\?"는 문자열의 ?로 인식하라고 하는 것!
    # ? 뒤의 어떠한 문자가 올때, 몇글자이던 상관이없다(갯수와 제한이 없음) '.*'
    # 그룹 바깥에 쓰여진 ? -> 있을 수도 있고 없을 수도 있다.
    # 특정 문자열의 마지막에 위치한다 => '$'
    
    ## 위와 같은 패턴을 가진 문자열을 찾아와서!((\d+)과 매칭되는것) m에 그룹화하는 것으로 반환해라! 만약(if) 없으면(else) None을 반환해라!
    return m.group(1) if m else None

### 두번쨰 함수! 완전한 url 만들기
def build_product_url(pid:str | None) -> str: # 여기서는 정규표현식의 or(\)과 다름 =|
    return f"https://www.wconcept.co.kr/Product/{pid}" if pid else "" #pid가 있다면 문자열을 반환해라, 그렇지 않다면 빈문자를 반환해라
    
### category_urls를 한번씩 반복해서 돌릴것!
for cat_name, url in category_urls.items() : # .items : 우리는 딕셔너리를 가져올 것이기 때문에 키값 / 밸류값을 나눠줄 필요가 있음. 이떄 사용하는 함수 
# cat_name => 키값 , url => 밸류값 이 각각 들어감
    driver.get(url)
    time.sleep(3)

    ##중첩 for문!
    ## 사이트의 방식이 최하단으로 가면 새로운 값을 주는 사이트에서 쓰이는 반복문 ( 반복해서 스크롤 내려야함 )
    for _ in range(10) : # 값을 찾아올 필요가 없음
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);") # 높이 설정 -> 매번 새로운 URL 들어갈 때마다 스크롤 내리기 / ; -> 자바스크립트의 구문 끝났다!
        time.sleep(2)

    ## 각각 카테고리마다 10번 스크롤하면서, 상품 찾아오기!
    products = driver.find_elements(By.CSS_SELECTOR,"div.product-item.item.type-all")[:100] 
    # 주의 !! 시간이 경과되면 소스코드가 바뀔 수가 있음! -> 안전하게 찾아오는 방법 : 안바뀔 것 같은것만 코드가져오기 / 100개 찾아와
    
    for product in products :
        try : # 실질적인 크롤링을하게되는 구문!
            ## 100개의 찾아온 것 중에 상품명을 제일 먼저것을 하나 찾아오고 name으로 저장해
            name = product.find_element(By.CSS_SELECTOR,"span.prdc-title span.text.detail").text.strip()# 상품명 & 브랜드 명이 필요할 수도 있으니까 prdc title에서부터 찾아옴
            try:
                img = product.find_element(By.CSS_SELECTOR,"button.area-img img") # button의area-img 안에 있는 img태그를 찾아와서
                src = img.get_attribute("src") #src만 찾아오고 싶어! . src에 넣어라
                pid = extract_product_id_from_imgsrc(src) # 위에 설정한 함수활용해서 결과값을 pid에 넣고
                url = build_product_url(pid) # 위에 설정한 pid를 넣어서 url로 완성시켜줘
            except: # 에러가 뭔지 알필요없는 상태, 만약 에러가 발생한다면, url =""인상태로 가져와!
                url = ""
                
            ##리뷰 카운트 수 찾아오기
            #괄호를 배제하는 방법 -> 정규표현식 쓰기!
            try:
                review_info = product.find_element(By.CSS_SELECTOR,"span.area-info span.text.cnt").text
                #위에서 가져온 것에서 (10)에서 숫자 가져오기! 정규표현식으로! -> 문자열로 가져와야함(정규표현식은)
                review_count = int(re.sub(r"[^\d]", "", review_info)) if review_info else 0 # review_info가 있을때만! 없으면 0으로 줘라. (SQL null값 들어오는거 방지)
                ## 교체하는 정규표현식의 형태
                # 패턴인 [] 안에 정수값을 넣을것.
                #^ 숫자가 아닌값을 찾아와라
                #, "" 앞에 것을 바꿔버리자
                # int(...) : 정수값으로 형변환해야함! (정규표현식 쓰려고 문자열로 사용했으니까) 
            except:
                review_count = 0
            ## 평점가져오기
            try :
                rating_text = product.find_element(By.CSS_SELECTOR,"span.area-info em.score").text
                # 여기서 찾아온 예) 4.9 라는 숫자는 소수점을 가진 숫자값으로 바꿔줘야함 (원래 문자열)
                review_avg = float(re.findall(r"[0-9.]+", rating_text)[0]) # 0번째 값으로 찾아와
                # 정규표현식으로 찾아오는것
                # (r"[0-9.]") : 숫자 0-9까지 들어올 수 있고, '.'이 들어올 수 있음! 여기서 '.'은 숫자 뒤에 들어왔으니까 실수로 인식한다는 문법!
                # rating_text에서 찾아오면 됨!
            except: #값을 못찾아왔다면
                review_avg = 0.0 #평점 0.0으로 줘라

            ## 데이터에 값넣기!
            all_data.append({  # 딕셔너리 형태로 넣을것!
                "카테고리":cat_name,
                "상품명":name,
                "URL":url,
                "리뷰수":review_count,
                "평균평점":review_avg
            }) 
            
        except Exception as e : #Exception이 발생하면 e를 출력하고
            print("에러발생 :", e)
            continue # 다시 이어가라!

driver.quit()

#print(all_data)
######################################################### PART 2 ########################################################
### df라는 변수에 판다스가 정리해준 all_date를 넣기!
df = pd.DataFrame(all_data)

## 리뷰수를 기준으로 상위 100개를 끊어내기!
df_top100 = df.sort_values(by = "리뷰수", ascending=False).head(100).reset_index(drop=True) #정렬 도와주는 함수 /
# by = sql의 group by 개념 (정렬 기준 컬럼 지정)
#ascending=False-> 내림차순정렬
#.head : 미리 확인할 때 사용하는 메서드. 앞에서 n개 행 출력(기본 5개).
# .reset_index(drop=True) -> index 값 새롭게 정렬 해주고 기존의 인덱스는 버려줘(drop)

## 카테고리별 평점이 몇점인지 알고싶어!
category_summary = df.groupby ("카테고리").agg({ # category 
    "리뷰수" : "sum", 
    "평균평점" : "mean"
}).reset_index() # 기존 category가 필요하기 때문에 drop쓰면 기존값이 안나옴.. 
# .groupby  = sql의 group by
# .agg : 집계함수를 쓰겠다!
# "sum", "mean" -> 합계, 평균
# 여기선 드랍 필요없음

## 출력
print(f"""
상위 100개 상품 :
{df_top100.head()}

카테고리별 통계:
{category_summary}
""")


상위 100개 상품 :
   카테고리                                    상품명  \
0    신발                OORIGINAL BLACK - 조리 블랙   
1    가방  [14%쿠폰] [단독] Panier Petit Bag_6colors   
2  액세서리   [단독] Round Logo Ball Cap (10 Colors)   
3  액세서리           [단독]BT8032 티디 스퀘어 버클 벨트 18mm   
4  여성의류                       실켓 코튼 티셔츠_8color   

                                            URL   리뷰수  평균평점  
0  https://www.wconcept.co.kr/Product/301721294  8478   5.0  
1  https://www.wconcept.co.kr/Product/301756891  3730   5.0  
2  https://www.wconcept.co.kr/Product/301628486  3682   4.9  
3  https://www.wconcept.co.kr/Product/304460283  3131   4.9  
4  https://www.wconcept.co.kr/Product/301218440  2968   4.9  

카테고리별 통계:
   카테고리    리뷰수   평균평점
0    가방  24218  4.267
1    신발  27342  4.281
2  액세서리  30519  4.016
3  여성의류  31521  4.435



# 엑셀로 가져가기

In [10]:
######################################################### PART 3 ########################################################
from openpyxl import Workbook

wb = Workbook()
#엑셀파일에서 첫번째 시트를 활성화시키겠다. 그리고 그 시트가 top100의 값을 넣을 수 있는 시트이다!
ws_top100 = wb.active

ws_top100.title = "상위100상품"
## 컬럼에 값주기
ws_top100.append(["카테고리","상품명","URL","리뷰수","평균평점"]) #[]에 순서대로 넣으면 컬럼에 순차적으로 들어감!

# dataFrame파일을 찾아와서 판다스에서 추출한 값을 넣기! 하나의 행! (0  https://www.wconcept.co.kr/Product/301721294  8478   5.0)
for idx,row in df_top100.iterrows(): #데이터 프레임에 있는 값을 넣기위해 각각의 행을 하나씩 찾아서 for in문 안에 있는 idx값에 넣을 수 있다.
    ws_top100.append([row["카테고리"],row["상품명"],row["URL"],row["리뷰수"],row["평균평점"]]) # 기존의 행안에 있었던 값안애

## 시트 하나 더 만들기
ws_category = wb.create_sheet(title = "카테고리별 통계")

ws_category.append(["카테고리","총 리뷰수","평균평점"])
#카테고리 섬머리에서 찾은 로우의 값을 찾을 떄마다 하나의 행에 넣는다
for idx, row in category_summary.iterrows() :
    ws_category.append([
        row["카테고리"],row["리뷰수"],round(row["평균평점"],2) # 소수점 두번째 자리까지 반올림(round("",2))
    ])

## 저장
filename = "wconcept_result.xlsx"
wb.save(filename) 
print(f"🫡1개의 엑셀 파일에 시트 2개 저장완료 : {filename}")

🫡1개의 엑셀 파일에 시트 2개 저장완료 : wconcept_result.xlsx


# 엑셀파일 SQL로 보내기

In [19]:
######################################################### PART 4 ########################################################
import pandas as pd
import pymysql
from sqlalchemy import create_engine, text # sqlalchemy 써보기 : text라는 함수를 통해서 create_engine을 활용해서 실제로 실행을 할것!

### 값을 넣을 DB 지정 (이미 DB가 만들어져 있어야 함.)
DB_NAME = "wconcept_db_v1"

### 저장할 테이블(TABLE) 이름 정의
TABLE_TOP100 = "top100_products"
TABLE_CATEGORY = "category_summary"

### 로컬호스트 접속
USER = "root"
PASSWORD = "asdf1122" #각자 비밀번호!
HOST ="127.0.0.1"
PORT = 3306

### 엑셀 파일 경로 지정
xlsx_path = "wconcept_result.xlsx"
## 뽑아올 데이터가 엑셀 시트 이름
SHEET_TOP100 = "상위100상품"
SHEET_CATEGORY = "카테고리별 통계"

### 찾아올 엑셀파일
df_top100 = pd.read_excel(xlsx_path, sheet_name=SHEET_TOP100)
df_category = pd.read_excel(xlsx_path, sheet_name=SHEET_CATEGORY)

### 영어로 바꾸기! 과정1 - 딕셔너리 형태로.
top100_cols = {
    "카테고리" : "category",
    "상품명" : "product_name",
    "URL" : "url",
    "리뷰수" : "review_count",
    "평균평점" : "avg_rating"
}

category_cols = {
    "카테고리" : "category",
    "총 리뷰수" : "total_review_count",
    "평균평점" : "avg_rating"
}

### 영어로 바꾸기! 과정2 - 값 재할당
df_top100 = df_top100.rename(columns=top100_cols)[list(top100_cols.values())] # df_top100을 가져와서 . 각 컬럼명을 바꾸겠다!
# 실제 값이 바뀌는 곳 [] / 각각의 값을 list() 안의 value들의 값을 바꾸겠다! / .values : 키값을 제외한 밸류 값들이 list()안에 쭉나열됨!!
# .rename() : 안에 들어가는 이름을 바꾸겠다

## 두번쨰 시트도 똑같이!
df_category = df_category.rename(columns=category_cols)[list(category_cols.values())] 

### null 형태의 값이 들어오면 제거하는것 (.dropna) = 값이 존재하는 요소만 가져와라!
df_top100 = df_top100.dropna(subset=["category","product_name","review_count","avg_rating"]) 
df_category = df_category.dropna(subset=["category","total_review_count","avg_rating"]) 
# subset : NaN 값을 검사할 특정 열 지정, 지정된 열 중 하나라도 NaN이면 해당 행 삭제
# URL 을 빼도 되는 이유 : 없는 경우 빈값을 넣었기 때문에 NULL이 나와서 값이 이상해질수도..

### type 정하기 
df_top100["review_count"] = df_top100["review_count"].astype(int) # .astype(): 특정 열의 속성을 문자열, 숫자열 등으로 가져올지 
df_top100["avg_rating"] = df_top100["avg_rating"].astype(float).round(2) # 실수로 가져오고 2번쨰 자리에서 반올림
df_category["total_review_count"] = df_category["total_review_count"].astype(int)
df_category["avg_rating"] = df_category["avg_rating"].astype(float).round(2)

### 이제 sql이랑 연결!!
# sqlalchemy 는 commit까지 완료시키는 기능이 있음! : autocommit=True -> 앞에 값을 다 넣은 후에 commit까지 자동으로!
conn_admin = pymysql.connect(host=HOST, user = USER, password = PASSWORD, port = PORT, autocommit=True)
## DB 만들기
with conn_admin.cursor() as cur :
    cur.execute(f"CREATE DATABASE IF NOT EXISTS `{DB_NAME}` CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;") 
    # .execute : mysql에 접속해서 컨트롤할 수 있는 기능을 부여받음
    # CHARACTER SET utf8mb4 -> mysql에 DB나 테이블을 넣어서 값을 넣으려고 할때 특수 문자들이 깨지는 경우 多 so, utf8mb4 쓰면 코딩과정 소실 잡아줌
    # COLLATE utf8mb4_unicode_ci; -> 이모지, 특수문자의 인코딩 디코딩 과정의 오류를 없애줌
    
### 닫아줌    
conn_admin.close()

# 엔진생성 왜?
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4" #해당 DB에 들어가기 위해 엔진을 작동시킴
) # ?charset=utf8mb4 -> 이 코드가 있어야 엔진을 작동시킬 수 있다!

### 테이블 만들기
# 만약에 wconcept_db_v1 과 같은 특수문자가 들어가있는 요소는 sql에서 오류발생 가능! ``으로 안전장치!
# 필드명 지정할때도 빽틱 (``)으로 안전장치!
ddl_top100 = f"""
CREATE TABLE IF NOT EXISTS `{TABLE_TOP100}` (
    `id` BIGINT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `category` VARCHAR(50) NOT NULL,
    `product_name` VARCHAR(255) NOT NULL,
    `url` TEXT NULL,
    `review_count` INT NOT NULL,
    `avg_rating` DECIMAL(3,2) NOT NULL,
    `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
)
"""
# DECIMAL(3,2) 총 3가지의 숫자로 이뤄진, 소수점 2번째 자리까지 값이 채워지는 숫자!

## 두번째 테이블
ddl_category = f"""
CREATE TABLE IF NOT EXISTS `{TABLE_CATEGORY}` (
    `id` INT UNSIGNED NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `category` VARCHAR(50) NOT NULL,
    `total_review_count` INT NOT NULL,
    `avg_rating` DECIMAL(3,2) NOT NULL,
    `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP
)
"""


### begin을 활용해서 엔진을 실행
with engine.begin() as conn :
    conn.execute(text(ddl_top100)) # top_100안에 들어가있는 문자열(""" """) 함수를 읽고 실행시킨다!
    conn.execute(text(ddl_category))
## 테이블에 값을 insert하기
with engine.begin() as conn :
    # [[]] : 하나의 행을 기준으로 각 열에 값이 하나씩 순서대로 들어오게 하기위해 예시만들기
    df_top100[["category","product_name","url","review_count","avg_rating"]].to_sql(  # mysql로 보내기
        TABLE_TOP100, con = conn, if_exists="append", index=False 
    ) 
    # TOP 100에 넣는다! 
    # con=conn : to_sql()이 데이터를 저장할 DB 연결(Connection)을 지정 
    # if_exists="append" : 이미 값이 있으면 추가하고 
    # index=False : 인덱스는 만들지 않는다!
    df_category[["category","total_review_count","avg_rating"]].to_sql( 
        TABLE_CATEGORY, con = conn, if_exists="append", index=False 
    )

###  코드를 잘못써서 my sql에 도달하지 못했을 경우를 대비한 코드
#()요소 : 어떠한 정보를 통해서 값을 전달할 것인지 넣음, 다음으로 어떠한 테이블에 값을 넣을것인지, 인덱스 이름을 준다. (반환값은 없음
def ensure_index(conn,schema, table, index_name,column_list) : 
    #쿼리부분을 사용해서 실행해볼 예정. / information_schema의 statistics속성안에 있는 기본 값인 1을 가져온다.
    q = text(""" 
        SELECT 1
        FROM information_schema.statistics
        WHERE table_schema =:schema AND
        table_name =:table AND
        index_name =:index_name
    """) # schema 탭 안에 들어가 있다면 (DB가 있는지 없는지 확인, 만들어졌다면 1을보여줘)
    #테이블의 이름이 있는지 확인하고,
    # index_name에 값이 있다면, 1을 가져와라!
    
    exists = conn.execute(q,{"schema": schema, "table": table, "index_name":index_name}).fetchone() # .fetchone(): 매칭이되면 exist를 출력할것.

## 위 함수 실행
with engine.begin() as conn:
    ensure_index(conn,DB_NAME,TABLE_TOP100,f"idx_{TABLE_TOP100}_category",["category"])  # 위 함수에서 설정한대로 나열.
    ensure_index(conn,DB_NAME,TABLE_TOP100,f"idx_{TABLE_TOP100}review_count",["review_count"])
    ensure_index(conn,DB_NAME,TABLE_TOP100,f"idx_{TABLE_TOP100}avg_rating",["avg_rating"])  
    ensure_index(conn,DB_NAME,TABLE_CATEGORY,f"idx_{TABLE_CATEGORY}total_review_count",["total_review_count"])  
    ensure_index(conn,DB_NAME,TABLE_CATEGORY,f"idx_{TABLE_CATEGORY}avg_rating",["avg_rating"])  
print("🫡MYSQL 데이터 생성 완료!🎇🎆")

🫡MYSQL 데이터 생성 완료!🎇🎆
